<img src="logo.png">

# Titanic

La tabla de ``Titanic.csv``, que puedes descargar [aquí](https://github.com/scidatmath2020/Miscelanea/blob/main/Python/Estadistica%20y%20Ciencia%20de%20datos/Titanic/titanic.csv), contiene información sobre la edad, el tipo de clase, sexo y superviviencia de las personas que estuvieron abordo del Titanic durante su desastrozo viaje.

Responde las siguientes preguntas:

1. Determina el número de supervivientes por sexo. ¿Las mujeres pasajeras tuvieron mayor oportunidad de sobrevivir que los hombres?

2. Determina el número de supervivientes por edad. ¿Los niños tuvieron mayor oportunidad de sobrevivir que los adultos?

3. Considera el número de supervivientes en función de la edad y el sexo. ¿El patrón observado en 1 se mantiene igual para niños y adultos?

4. La documentación del viaje del Titanic sugiere que la política *niños y mujeres primero* no fue "enteramente exitosa" para salvar a niños y mujeres de la 3a clase. Verfica esto haciendo un gráfico de barras.

In [ ]:
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
from plotnine import *

In [ ]:
titanic = pd.read_csv("titanic.csv")

In [ ]:
titanic

In [ ]:
titanic >> filter(_.Survived == "Yes") >> group_by(_.Sex) >> summarize(total_survived = _.n.sum())

In [ ]:
titanic >> mutate(survived_code = np.where(_.Survived == "Yes",1,0))

In [ ]:
titanic = titanic >> mutate(survived_code = np.where(_.Survived == "Yes",1,0))

In [ ]:
titanic >> filter(_.Sex=="Female")

In [ ]:
(titanic >> group_by(_.Sex) >> summarize(total_travelers=_.n.sum(),
                                        total_survived = (_.n * _.survived_code).sum(), 
                                        per = 100*(_.n * _.survived_code).sum()/_.n.sum())
)

In [ ]:
(titanic >> group_by(_.Age) >> summarize(total_travelers=_.n.sum(),
                                        total_survived = (_.n * _.survived_code).sum(), 
                                        per = 100*(_.n * _.survived_code).sum()/_.n.sum())
)

In [ ]:
(titanic >> group_by(_.Age,_.Sex) >> summarize(total_travelers=_.n.sum(),
                                        total_survived = (_.n * _.survived_code).sum(), 
                                        per = 100*(_.n * _.survived_code).sum()/_.n.sum())
)

In [ ]:
resumen = (titanic >> group_by(_.Age,_.Sex,_.Class) >> summarize(total_travelers=_.n.sum(),
                                        total_survived = (_.n * _.survived_code).sum(), 
                                        per = 100*(_.n * _.survived_code).sum()/_.n.sum())
)

In [ ]:
resumen

In [ ]:
(ggplot() +
     geom_col(data = resumen,
              mapping = aes(x="Class",y="per",fill="Sex"),
              position = "dodge",
              alpha=0.5
             ) +
     geom_col(data = resumen, 
              mapping = aes(x="Class", y="per-2",fill="Sex"),
              position="dodge",
              width = 0.8
              ) +
 facet_grid("Age~.") +
 scale_fill_manual(values = ["#009E73","#E69F00"]) +
    theme(panel_background = element_rect("#202020"),
          panel_grid = element_line(color="darkgrey",size=0.1)) 
)